In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold, StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
import category_encoders as ce

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import Lars
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from IPython.display import Image

import category_encoders as ce
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import ShuffleSplit

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

from sklearn.ensemble import GradientBoostingRegressor

from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.feature_selection  import RFE
from sklearn.decomposition import PCA


### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [5]:
# Путь к тренировочному набору
path_train = 'train_house.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test_house.csv' # содержит только имя файла, без имен папок

In [6]:
# Блок(и) обучения и поверки модели

In [7]:
df = pd.read_csv(path_train)

X = df.drop(['price'],axis=1)
y = df['price']


df[0]

df.loc[0]

df.get[0]

df.rows[0]

In [27]:
df.loc[0]

price                     3450
m2                     8.19929
beds                       2.0
bath                       2.0
laundry            (b) on-site
pets               (d) no pets
housing_type         (c) multi
parking          (b) protected
hood_district              8.0
Name: 0, dtype: object

In [8]:
axis_features = ['m2', 'beds','bath','hood_district']

imputer_for_nums = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors = 5, missing_values=np.nan)),
    ('scaler', MinMaxScaler())
])

imputer_for_categorical_OHE = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),   
    ('scaler', OneHotEncoder())
    
])


imputer_for_categorical_ordinal = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', OrdinalEncoder())
    
])

order_for_cat = [{
    'col':'pets',
    'mapping':{'(d) no pets':0, '(c) cats':1, '(b) dogs':2, '(a) both':3}
    }]

imputer_for_categorical_with_order = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=order_for_cat)),
    ('scaler', MinMaxScaler())
    ])


order_for_cat2 = [{
    'col':'housing_type',
    'mapping':{'(c) multi':0, '(b) double':1, '(a) single':2}
    }]

imputer_for_categorical_with_order2 = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ce',ce.OrdinalEncoder(mapping=order_for_cat2)),
    ('scaler', MinMaxScaler())
    ])


order_for_cat3 = [{
    'col':'laundry',  
    'mapping': {'(b) on-site':0, '(a) in-unit':1, '(c) no laundry':2}
    
}]

imputer_for_categorical_with_order3 = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=order_for_cat3)),
    ('scaler', MinMaxScaler())
    ])

order_for_cat4 = [{
    'col':'parking',
    'mapping':{'(d) no parking':0, '(c) off-street':1, '(b) protected':2, '(a) valet':3}
    }]

imputer_for_categorical_with_order4 = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=order_for_cat4))
    ])


CT = ColumnTransformer([
    
        ("imputer_for_nums", imputer_for_nums, axis_features), # 
        ("imputer_for_categorical_with_order", imputer_for_categorical_with_order, ["pets"]), # приоритизация
         ("imputer_for_categorical_with_order2", imputer_for_categorical_with_order2, ["housing_type"]),
        ("imputer_for_categorical_with_order3", imputer_for_categorical_with_order3, ["laundry"]),
         ("imputer_for_categorical_with_order4", imputer_for_categorical_with_order4, ["parking"]),
             ], 
    
    
        remainder = 'passthrough')

display(CT)

res_ct = CT.fit_transform(X)

ColumnTransformer(remainder='passthrough',
                  transformers=[('imputer_for_nums',
                                 Pipeline(steps=[('imputer', KNNImputer()),
                                                 ('scaler', MinMaxScaler())]),
                                 ['m2', 'beds', 'bath', 'hood_district']),
                                ('imputer_for_categorical_with_order',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'pets',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': (d) no pets    0
(c) cats       1
(b) dogs       2
(a) both       3
dt...
                                                  OrdinalEncoder(mapping=[{'col': 'laundry',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': (b) on-site       0
(a) in-unit       1
(c) no laundry    2
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['laundry']),
                                ('imputer_for_categorical_with_order4',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'parking',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': (d) no parking    0
(c) off-street    1
(b) protected     2
(a) valet         3
dtype: int64}]))]),
                                 ['parking'])])

In [9]:
# Блок предсказания с использованием тестового набора

In [10]:
X_testoviy = pd.read_csv(path_test)
X_testoviy.head(5)

,m2,beds,bath,laundry,pets,housing_type,parking,hood_district
0,6.749310,1.0,1.0,(b) on-site,(b) dogs,(c) multi,(b) protected,8.0
1,5.782657,1.0,1.0,(a) in-unit,(c) cats,(c) multi,(d) no parking,9.0
2,4.315416,1.0,1.0,(b) on-site,(a) both,(c) multi,(b) protected,2.0
3,9.010588,2.0,2.0,(a) in-unit,(a) both,(c) multi,(b) protected,9.0
4,3.883874,0.0,1.0,(b) on-site,(a) both,(c) multi,(d) no parking,8.0


In [11]:
pipe_1 = Pipeline([('preprocessing', CT),
                ('regression', RandomForestRegressor(n_estimators= 150, max_depth = 6, max_features = 0.4, min_samples_leaf = 1, 
                                                     max_samples = 0.5 ,n_jobs = -1))])
pipe_1.fit(X,y)
y_predict = pipe_1.predict(X_testoviy)

print("Train:",mean_absolute_percentage_error(y, pipe_1.predict(X)))
# print("Test:",mean_absolute_percentage_error(y_test, y_pred_1))

Train: 0.11177273447845161


In [12]:
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict

array([3076.1218355 , 2793.71191754, 2514.40367632, 3894.71596437,
       2005.19991089, 4567.21070552, 3732.53620603, 3517.12015628,
       3303.40164183, 4202.58308338, 3152.52150333, 2835.81892057,
       3275.76725896, 3285.20511131, 1937.10783469, 4807.31709424,
       2488.24800046, 4714.77407657, 3736.38848403, 4401.61891094,
       4247.12885477, 4123.41601382, 3762.39413746, 4951.16664723,
       3462.67923181, 2739.59340295, 4461.48294341, 3216.93206208,
       4654.76908684, 2301.9189306 , 2745.92882993, 2070.40288095,
       4384.76515667, 2681.42077041, 2048.42886146, 4730.16450155,
       4240.4053622 , 2725.507979  , 3450.58127418, 3358.09245468,
       2924.65905254, 2185.62293825, 2118.04963924, 3590.49925099,
       3023.60714189, 2758.59448269, 4482.25954756, 2700.85645934,
       2807.63672421, 3331.81111293, 2347.80728129, 3874.06876078,
       1728.05765775, 2829.26973611, 2844.62830339, 2732.04719917,
       4719.61835381, 2367.80840037, 4102.02893924, 2217.62217